In [ ]:
import os
from dotenv import load_dotenv
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.llms.cohere import Cohere
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
    Document
)
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.readers.file import CSVReader
from pinecone import Pinecone, ServerlessSpec
import pandas as pd

In [ ]:
load_dotenv()

In [ ]:
Settings.llm = Cohere(
        api_key=os.getenv("COHERE_API_KEY"),
        model="command-r",
    )

In [ ]:
Settings.embed_model = CohereEmbedding(
        cohere_api_key=os.getenv("COHERE_API_KEY"),
        model_name="embed-english-v3.0",
        input_type="search_document",
    )

In [ ]:
data = pd.read_csv("./data/04-03-003.csv")
data

In [ ]:
docs = [Document(text=d) for d in data["text"]] 
docs

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
        name="dogmatics",
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )

In [ ]:
pinecone_index = pc.Index("dogmatics")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, show_progress=True)


In [ ]:
 # create a cohere reranker
cohere_rerank = CohereRerank(api_key=os.getenv("COHERE_API_KEY"), top_n=4, model="rerank-english-v3.0")

In [ ]:
# create the query engine
query_engine = index.as_query_engine(node_postprocessors=[cohere_rerank])

In [ ]:
response = query_engine.query("Are there any mediators between God and man?")
print(response)